In [1]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("cssa_data.csv")

# 显示前几条数据
print(df.head())


        question                                         answer
0      如何加入CSSA？              你可以关注CSSA的微信公众号，或者在社交平台上联系工作人员报名。
1  墨尔本大学有哪些选课建议？                  建议选择自己感兴趣的课程，同时查看课程评分和往年考试难度。
2   CSSA会举办哪些活动？                  CSSA定期举办迎新会、联谊活动、学术讲座、求职分享会等。
3       墨尔本如何租房？  可以通过Facebook群组、Flatmates、CSSA租房信息群等渠道寻找合适的房源。


In [2]:
from datasets import Dataset

# 加载 CSV 数据集
dataset = Dataset.from_pandas(df)

# 查看数据格式
print(dataset)


d:\Anaconda3\envs\CSSA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'answer'],
    num_rows: 4
})


In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


In [ ]:
import faiss
import numpy as np
import pandas as pd
import torch
import gc
from transformers import AutoTokenizer, AutoModel

# ✅ 1️⃣ 读取小数据集
file_path = os.path.abspath("cssa_data.csv")
df = pd.read_csv(file_path)

# ✅ 2️⃣ 加载 Transformer 模型（更轻量级）
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

# 设置设备（确保只使用 CPU，避免 GPU 相关问题）
device = torch.device("cpu")
model.to(device)

# ✅ 3️⃣ 计算文本嵌入（仅 4 条数据，无需批量处理）
def embed_text(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return np.zeros((384,), dtype=np.float32)  # 返回固定维度的零向量

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=32).to(device)

    with torch.no_grad():  
        output = model(**inputs)

    return output.last_hidden_state.mean(dim=1).cpu().numpy().flatten().astype(np.float32)  # 保证是 (384,)

# ✅ 4️⃣ 创建 FAISS 索引
dimension = 384
index = faiss.IndexFlatL2(dimension)

# ✅ 5️⃣ 计算 4 条数据的向量，并存入 FAISS
question_vectors = np.array([embed_text(q) for q in df["question"]], dtype=np.float32)
index.add(question_vectors)  

print(f"✅ FAISS 向量索引创建完成！索引大小: {index.ntotal}")

# ✅ 6️⃣ 释放 Python 内存，防止 Jupyter 内核崩溃
gc.collect()
torch.cuda.empty_cache()


✅ FAISS 向量索引创建完成！索引大小: 4


In [10]:

# ✅ 7️⃣ 定义一个检索函数
def retrieve_answer(user_query):
    if index.ntotal == 0:
        return "❌ 错误: FAISS 索引为空，无法查询答案！"

    query_vector = embed_text(user_query).reshape(1, -1).astype(np.float32)  # 转换 float32
    distances, indices = index.search(query_vector, k=1)  

    if indices[0][0] == -1:
        return "❌ 错误: 找不到匹配的答案！"

    best_match = df.iloc[indices[0][0]]  
    return best_match["answer"]


In [12]:

# ✅ 8️⃣ 测试查询
query = "CSSA是什么"
response = retrieve_answer(query)
print(f"🔍 查询: {query}\n📌 最佳匹配答案: {response}")


🔍 查询: CSSA是什么
📌 最佳匹配答案: 你可以关注CSSA的微信公众号，或者在社交平台上联系工作人员报名。
